## Analyzing Customer Data to Indentify Best Churn Customer

There is a coupon of $1000 to be given to the best churned customers to woo them. I want to find the best churned cutomers to give it to from the customer list provided. A churned customer is one who hasn't purchased anything for a while. I have defined the churned period to be the past three months. 

Use group_by_customer together with an appropriate aggregating method to get the total amount spent by each customer as a column in best_churn called amount_spent. Drop the trans_date column

In [1]:
#Importing packages and loading the data set
import pandas as pd
import datetime as dt
data = pd.read_csv('rfm_xmas19.txt', parse_dates=["trans_date"])

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   customer_id  125000 non-null  object        
 1   trans_date   125000 non-null  datetime64[ns]
 2   tran_amount  125000 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.9+ MB


Use group_by_customer together with an appropriate aggregating method to get the total amount spent by each customer as a column in best_churn called amount_spent and drop the trans_date column

In [3]:
group_by_customer = data.groupby("customer_id")
last_transaction = group_by_customer["trans_date"].max()
best_churn = pd.DataFrame(last_transaction)

cutoff_day = dt.datetime(2019, 10, 16)

best_churn["churned"] = best_churn["trans_date"].apply(lambda date: 1 if date < cutoff_day else 0)
best_churn

,trans_date,churned
customer_id,,
FM1112,2019-10-14,1
FM1113,2019-11-09,0
FM1114,2019-11-12,0
FM1115,2019-12-05,0
FM1116,2019-05-25,1
...,...,...
FM8996,2019-09-09,1
FM8997,2019-03-28,1
FM8998,2019-09-22,1


In [4]:
best_churn["nr_of_transactions"] = group_by_customer.size()
best_churn["amount_spent"] = group_by_customer.sum()
best_churn.drop("trans_date", axis="columns", inplace=True)

In [5]:
best_churn

,churned,nr_of_transactions,amount_spent
customer_id,,,
FM1112,1,15,1012
FM1113,0,20,1490
FM1114,0,19,1432
FM1115,0,22,1659
FM1116,1,13,857
...,...,...,...
FM8996,1,13,582
FM8997,1,14,543
FM8998,1,13,624


Lets find the best customers based on amount spent and number of purchases made. We decide to use a very simple weighted sum model to classify customers.(0.5 x amount spent)+(0.5 x purchases made)

We saw that if a customer has made two purchases totalling 500 dollar, their score would be . If a customer has spent $400 across 20 different purchases, their score would be .

The second customer is clearly a regular customer and shows potential to spend more than the first in the long run, but it is scored lower than the first. This score with the data as is, favours money spent over the number of purchases. This happens for two reasons:

- Money spent is usually much higher than the number of transactions;
- We are using the same weight (0.5) for both criteria. 

We'll fix this by appropriately modifying the data. We'll use a technique called min-max feature scaling. (X - min(X))/(max(X) - min(X))

The min-max scaling technique will rescale both columns so that we can make an apples-to-apples comparison. It is appropriate for data with irregular scaling


In [6]:
best_churn["scaled_tran"] = (best_churn["nr_of_transactions"] - best_churn["nr_of_transactions"].min()) \
                             / (best_churn["nr_of_transactions"].max() - best_churn["nr_of_transactions"].min())

best_churn["scaled_amount"] = (best_churn["amount_spent"] - best_churn["amount_spent"].min()) \
                               / (best_churn["amount_spent"].max() - best_churn["amount_spent"].min())

best_churn["score"] = 100*(.5*best_churn["scaled_tran"] + .5*best_churn["scaled_amount"])
best_churn.head()

,churned,nr_of_transactions,amount_spent,scaled_tran,scaled_amount,score
customer_id,,,,,,
FM1112,1,15,1012,0.314286,0.309986,31.213567
FM1113,0,20,1490,0.457143,0.481681,46.941195
FM1114,0,19,1432,0.428571,0.460848,44.470956
FM1115,0,22,1659,0.514286,0.542385,52.833539
FM1116,1,13,857,0.257143,0.254310,25.572660


Now that we have a way to compare customers, we need to decide on a threshold to determine which customers are "the best." Should it be the first 20 customers? The first 40 customers? The top 10% of customers? What are the criteria?

We could use advanced techniques like k-means clustering (which you'll learn about later), hierarchical clustering, or employ some other machine learning algorithm, but that would take a lot of time.

With all this in mind, you decide to employ the following strategy to determine the cutoff point:

- Find the mean of the transactions and compute 30% of that. Make this the value of the coupon;
- Divide the budget by the value obtained above to get the number of coupons you're going to be sending out;
- Pick the first  churned customers where  is the result of the calculation done in the previous step. This is your cutoff point

In [7]:
coupon = data["tran_amount"].mean()*0.3
nr_of_customers = 1000/coupon
print(coupon, nr_of_customers, sep="\n")

19.4975736
51.28843314123969


Due to biological, physical, and conceptual reasons, we can't have 51 and a bit customers. Moreover, you deem that a value of roughly 19.50 dollar is a weird value for a coupon, so you round it to $20, and then decide to send the coupon to the top 50 churned customers. Once you complete this urgent request, you email your manager the text file and bring up the following points:

- Given the budget, you decided to send $20 coupons to the 50 best customers.
- A brief mention that you ranked customers on number of purchases made and amount spent — without getting into too much detail.
- The deliverable has more than just the customer IDs, but the recipients should not worry about the other columns.

In [8]:
#saving the filtered to 50 customers to a csv file
top_50_churned = best_churn.loc[best_churn["churned"] == 1].head(50)

top_50_churned.to_csv("best_customers.txt")